## Problem 1

In [1]:
import numpy as np
from scipy import linalg as la

def is_drazin(A, D, k):
    product1 = np.dot(A,D)
    product2 = np.dot(D,A)
    exp1 = np.linalg.matrix_power(A, k+1)
    exp0 = np.linalg.matrix_power(A, k)
    product3 = np.dot(exp1, D)
    product4 = np.dot(D,product1)
    
    if np.allclose(product1, product2) is False:
        return False
    elif np.allclose(product3, exp0) is False:
        return False
    elif np.allclose(product4, D) is False:
        return False
    else:
        return True
    

In [2]:
#Test 1

A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
D = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])

is_drazin(A,D,1)

True

In [3]:
#Test 2

B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
D = np.zeros((3,3))

is_drazin(B,D,3)

True

In [4]:
#Test 3

B = np.array([[3,1,4],[5,2,6],[-2,-1,-3]])
D = np.zeros((3,3))

is_drazin(B,D,3)

False

## Problem 2

In [5]:
def drazin(A, tol):
    """Computes the Drazin inverse of an nxn matrix A"""
    n = np.shape(A)[0]
    f = lambda x: abs(x) > tol
    Q1,S,k1 = la.schur(A, sort=f)
    f = lambda x: abs(x) <= tol
    Q2,T,k2 = la.schur(A, sort=f)
    
    U = np.concatenate((S[:,:k1],T[:,:n-k1]),axis=1)
    Uinv = np.linalg.inv(U)
    V = np.dot(np.dot(Uinv, A), U)
    Z = np.zeros((n,n))
    
    if k1 != 0:
        Minv = np.linalg.inv(V[:k1,:k1])
        Z[:k1,:k1] = Minv
        
    return np.dot(np.dot(U,Z),Uinv)
    

In [6]:
#Testing Function

A = np.random.random((4,4))
D = drazin(A, 0.01)

is_drazin(A,D,2)

True

## Problem 3

In [85]:
from scipy.sparse import csgraph

def resistance(A):
    """Accepts an nxn matrix of an undirected graph
    Returns the effective resistance
    Requires the csgraph package from scipy.spare
    """
    L = csgraph.laplacian(A)
    n = np.shape(A)[0]
    R = np.zeros((n,n))
    i = 0
    while i < n:
        L_star = np.copy(L)
        identity = np.zeros(n)
        identity[i] = 1
        L_star[i] = identity
        L_drazin = drazin(L_star, 0.001)
        R[:,i] = np.diag(L_drazin)
        R[i,i] = 0
        i +=1
    return R

In [86]:
#Test Case 1: Upper Left Example
A = np.array([[1,1,0,0],[1,1,1,0],[0,1,1,1],[0,0,1,1]])
print(resistance(A))

[[ 0.  1.  2.  3.]
 [ 1.  0.  1.  2.]
 [ 2.  1.  0.  1.]
 [ 3.  2.  1.  0.]]


In [87]:
#Test Case 2: Triangle
A = np.array([[1,1,1],[1,1,1],[1,1,1]])
print(resistance(A))

[[ 0.          0.66666667  0.66666667]
 [ 0.66666667  0.          0.66666667]
 [ 0.66666667  0.66666667  0.        ]]


The output matches the test cases chosen so it appears that our function works!

## Problems 4 and 5

In [197]:
import pandas as pd

class LinkPredictor:
    def __init__(self, filename):
        
        #Initialize the list of indices
        a = []
        b = []
        with open(filename, 'r') as file:
            connections = file.readlines()
        for entry in connections:
            connection = entry.split(',')
            a.append(connection[0])
            b.append(connection[1].strip())
        names = np.column_stack((a,b))
        m = np.shape(names)[0]
        n = np.shape(names)[1] 
        person, pos = np.unique(names, return_inverse=True)
        pos = pos.reshape((m,n))
        
        #Set up the network 
        k = len(person)
        network = np.zeros([k,k], dtype=float)
        i = 0
        while i < m:
            network[pos[i,0],pos[i,0]] = 1
            network[pos[i,0],pos[i,1]] = 1
            network[pos[i,1],pos[i,0]] = 1
            i += 1
        
        #Calculate connections and effective resistance
        self.A = network.astype(float)
        self.R = resistance(network)
        self.people = person
            
    #Problem 5 functions
    def predict_link(self,node=None):
        R2 = np.copy(self.R)
        R2=R2*(self.A==0)
        R2[R2==0]=np.max(R2)
        if node is None:
            minimum = np.min(R2)
            loc = np.where(R2==minimum)
            return (self.people[loc[0][0]],self.people[loc[1][0]])
        else:
            if node not in self.people:
                raise ValueError("You must enter a valid name.")
            else:
                position = np.where(self.people==node)
                column = R2[:,position]
                minimum = np.min(column)
                loc = np.where(column==minimum)
                return self.people[loc[0][0]]
        
    def add_link(self, node1, node2):
        if node1 not in self.people:
            raise ValueError("The first name entered is not valid.")
        elif node2 not in self.people:
            raise ValueError("The second name entered is not valid.")
        else:
            pos1 = np.where(self.people==node1)
            pos2 = np.where(self.people==node2)
            self.A[pos1, pos2] = 1
            self.A[pos2, pos1] = 1
            self.R = resistance(self.A)
            print("Connected!")

In [198]:
network = LinkPredictor('social_network.csv')
network.predict_link()

('Oliver', 'Emily')

In [199]:
network.predict_link("Melanie")

'Carol'

In [201]:
print(network.predict_link("Alan"))
network.add_link("Alan","Sonia")

Sonia
Connected!


In [202]:
print(network.predict_link("Alan"))
network.add_link("Alan","Piers")

Piers
Connected!


In [203]:
print(network.predict_link("Alan"))
network.add_link("Alan","Abigail")

Abigail
Connected!
